# Single-node training example

In [ ]:
# Pick up the local code
import sys
sys.path.append('..')

In [ ]:
# Externals
import keras
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

# Locals
from data import get_datasets
from models import get_model
from utils.device import configure_session
from utils.optimizers import get_optimizer
from utils.callbacks import TimingCallback

In [ ]:
%matplotlib inline

## Introduction

We will be doing image classification on CIFAR10:
https://www.cs.toronto.edu/~kriz/cifar.html

We will be training a simple CNN model to classify small images into 10 classes.

Let's start by looking at some example images from the dataset.

In [ ]:
x, y = cifar10.load_data()[0]

nrows = 8
ncols = 8

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 12), sharex=True, sharey=True)

for i, ax in enumerate(axs.flatten()):
    ax.imshow(x[i])
    
plt.tight_layout()

## Configuration

Here is where we will specify all of our configuration of the model, optimizer, and training procedure.

I'll also set some hardware related settings.

In [ ]:
# Model config
model_name = 'cnn'
input_shape = [32, 32, 3]
n_classes = 10
dropout = 0.1

# Optimizer config
optimizer_name = 'Adam'
lr = 0.001

# Training config
batch_size = 32
n_epochs = 32
loss_name = 'categorical_crossentropy'
metrics = ['accuracy']

In [ ]:
# Hardware config
gpu = 0
intra_threads = 2 # CPU performance knob
inter_threads = 2 # CPU performance knob

In [ ]:
configure_session(gpu=gpu, intra_threads=intra_threads, inter_threads=inter_threads)

## Load the data

Here we use the local get_datasets code to retrieve and configure our data.

This actually returns data generator iterators which can read data and perform on-the-fly augmentation.

In [ ]:
# Load the data
train_gen, valid_gen = get_datasets(name='cifar10', batch_size=batch_size)
train_steps = len(train_gen) // batch_size
valid_steps = len(valid_gen) // batch_size

## Build the model

Now we use our local get_model code to build our CNN model according to our configuration.

In [ ]:
# Build the model and optimizer
model = get_model(name=model_name, input_shape=input_shape, n_classes=n_classes, dropout=dropout)
opt = get_optimizer(name=optimizer_name, lr=lr, n_ranks=1)
model.compile(loss=loss_name, optimizer=opt, metrics=metrics)
model.summary()

## Train the model

We use the fit_generator method to train our CNN model on our data generators.

Watch the progess as our model eats through the training data and regularly evaluates on the validation data.

In [ ]:
# Train the model
history = model.fit_generator(train_gen,
                              epochs=n_epochs,
                              steps_per_epoch=train_steps,
                              validation_data=valid_gen,
                              validation_steps=valid_steps)

## Visualize training history

Training has now completed. We can use the returned history object to make plots of the training and validation set losses and accuracies during training. This is very valuable for identifying issues like under/over fitting!

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(12, 5))

# Plot the loss
ax0.plot(history.epoch, history.history['loss'], label='train')
ax0.plot(history.epoch, history.history['val_loss'], label='validation')
ax0.set_xlabel('Epoch')
ax0.set_ylabel('Loss')
ax0.legend(loc=0)

# Plot the accuracy
ax1.plot(history.epoch, history.history['acc'], label='train')
ax1.plot(history.epoch, history.history['val_acc'], label='validation')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(bottom=0, top=1)
ax1.legend(loc=0)

#prettify layout
plt.tight_layout()

## What to try next?

Now that you've gotten this far, familiarize yourself with what you can change in the configuration and the effect it has on model performance.

For example:
- What happens if you increase or decrease the learning rate by a factor of 10?
- What happens if you greatly increase or decrease the size of the model in number and size of layers?